In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import sys

sys.path.append("/home/dchen/BA_CH_EN/")

# Basics
import tensorflow as tf
import xarray as xr
import pandas as pd

# Helpful
from tqdm import tqdm

# Visualization
import matplotlib.pyplot as plt

# My Methods
from src.utils.CRPS import *  # CRPS metrics
from src.utils.data_split import *  # Splitting data into X and y
from src.utils.drn_make_X_array import *  # Import make train array functions (make_X_array)
from src.models.EMOS import *  # EMOS implementation
from src.models.DRN.DRN_model import *  # DRN implementation
from src.models.EMOS_global.EMOS_global_load_score import *  # Load EMOS_global_scores
from src.models.EMOS_global.EMOS_global_load_model import *  # Load EMOS_global_models
import data.raw.load_data_raw as ldr  # Load raw data
import data.processed.load_data_processed as ldp  # Load processed data normed
import data.processed.load_data_processed_denormed as ldpd  # Load processed data denormed
from src.models.CRPS_baseline.CRPS_load import *  # Load CRPS scores
from src.models.U_net.unet import Unet
from src.models.U_net.u_net_tests_params import *
from src.models.U_net.u_net_load_data import *

2023-07-04 17:39:16.440539: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/anaconda3/lib/
2023-07-04 17:39:16.440560: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


<IPython.core.display.Javascript object>

### 0. Setup

In [3]:
var_num = 2
lead_time = 0

<IPython.core.display.Javascript object>

In [4]:
def pad_land_sea_mask(land_sea_mask, desired_shape=(128, 144)):
    # The original land-sea mask has only spatial dimensions.
    # We'll add two extra dimensions at the beginning and the end to match the shape expected by the padding function.
    land_sea_mask = land_sea_mask[np.newaxis, ..., np.newaxis]

    # Apply padding
    padded_mask = pad_images(land_sea_mask, desired_shape)

    # Remove the extra dimensions we added earlier
    padded_mask = np.squeeze(padded_mask)

    return padded_mask

<IPython.core.display.Javascript object>

### 1. Prepare Data

In [5]:
land_sea_mask_dummy = np.load(
    "/Data/Delong_BA_Data/land_sea_mask_dummy/land_sea_mask_dummy.npy"
)
land_sea_mask_dummy = pad_land_sea_mask(land_sea_mask_dummy)
land_sea_mask_dummy = np.repeat(land_sea_mask_dummy[np.newaxis, ...], 1429, axis=0)

<IPython.core.display.Javascript object>

In [6]:
train_var_mean = []
train_var_std = []
train_var_y = []
for var in range(6):
    (
        padded_train_data_mean,
        padded_train_data_std,
        padded_train_data_y,
    ) = u_net_load_train_data(var, 0)
    train_var_mean.append(padded_train_data_mean)
    train_var_std.append(padded_train_data_std)
    train_var_y.append(padded_train_data_y)

<IPython.core.display.Javascript object>

In [7]:
# Then, pack all of your input data into a list
train_inputs = train_var_mean + train_var_std + [land_sea_mask_dummy]

train_target = train_var_y[var_num]

<IPython.core.display.Javascript object>

In [8]:
for i in range(13):
    print(train_inputs[i].shape)

(1429, 128, 144, 1)
(1429, 128, 144, 1)
(1429, 128, 144, 1)
(1429, 128, 144, 1)
(1429, 128, 144, 1)
(1429, 128, 144, 1)
(1429, 128, 144, 1)
(1429, 128, 144, 1)
(1429, 128, 144, 1)
(1429, 128, 144, 1)
(1429, 128, 144, 1)
(1429, 128, 144, 1)
(1429, 128, 144)


<IPython.core.display.Javascript object>

In [9]:
train_target.shape

(1429, 128, 144)

<IPython.core.display.Javascript object>

### 2. Build and Train Model

In [10]:
# Parameters for Unet class initialization
v = "tp"  # Or any other value based on your preference
train_patches = False  # Or True if you want to train with patches

# Initialize the U-Net model
unet_model = Unet(v=v, train_patches=train_patches, filters=16)

# Build the model with your training data shape
model = unet_model.build_model(padded_train_data_mean.shape, 2, learning_rate=0.01)

Downsampling block 1 shape:  (None, 64, 72, 64)
Downsampling block 2 shape:  (None, 32, 36, 128)


2023-07-04 17:39:26.422462: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/anaconda3/lib/
2023-07-04 17:39:26.422605: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/anaconda3/lib/
2023-07-04 17:39:26.422719: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/anaconda3/lib/
2023-07-04 17:39:26.422828: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file:

Downsampling block 3 shape:  (None, 16, 18, 256)
Downsampling block 4 shape:  (None, 8, 9, 512)
Bottleneck shape:  (None, 8, 9, 1024)
Upsampling block 5 shape:  (None, 8, 9, 1024)
Upsampling block 4 shape:  (None, 16, 18, 512)
Upsampling block 3 shape:  (None, 32, 36, 256)
Upsampling block 2 shape:  (None, 64, 72, 128)
Upsampling block 1 shape:  (None, 128, 144, 64)


<IPython.core.display.Javascript object>

In [11]:
hist = model.fit(
    train_inputs,
    train_target,
    epochs=150,
    shuffle=True,
    batch_size=128,
    validation_split=0.25,
)

Epoch 1/150
Var equals 0
Var equals 0
9/9 [==============================] - 210s 23s/step - loss: 281.4494 - val_loss: 1222441.6250
Epoch 2/150
9/9 [==============================] - 207s 23s/step - loss: 277.3220 - val_loss: 64819.5820
Epoch 3/150
9/9 [==============================] - 206s 23s/step - loss: 271.6201 - val_loss: 5954.4907
Epoch 4/150
5/9 [===============>..............] - ETA: 1:32 - loss: 265.5911

KeyboardInterrupt: 

<IPython.core.display.Javascript object>